# Check for Curation Attacks

In [1]:
# standard imports
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# function used to query each type of event from The Graph
def query_theGraph_subgraph(raw_query, field_name, url, verbose=False):

    query_parts =raw_query.split(')')
    paginator = ", where: {}" 
    n = 0
    records = []
    while n < 1000:
        if (n == 0):
            # order by blockNumber to get all events, as deafult ordering is by id
            query = 'query '+query_parts[0] + 'orderBy:createdAt, where:{createdAt_gt: 0})' + query_parts[1]
        else:
            query = 'query '+query_parts[0]+ 'orderBy:createdAt, where: {createdAt_gt: ' + str(indexer) + '})'+query_parts[1]

        if verbose:
            print(query)

        r = requests.post(url, json = {'query':query})

        try:
            d = json.loads(r.content)['data'][field_name]
        except:
            print(r.content)
            errors = json.loads(r.content)['errors']
            print(errors)
            for e in errors:
                print(e['message'])

        #print(f'results {len(d)}')
        records.extend(d)
        #print(f'total {len(records)}')
        try:
            indexer = d[-1]['createdAt']
        except:
            print('noblock')
        
        n += 1
        if len(d) < 99:
            break
        
    return pd.DataFrame(records)

In [4]:
#get subgraphs data
url = 'https://api.thegraph.com/subgraphs/name/graphprotocol/graph-network-mainnet'

subgraph_query = '''query{
	subgraphs(){
    id
    owner{
      id
    }
    currentVersion{
      id
      subgraphDeployment{
        id
      }
      version
    }
    createdAt
    updatedAt
    active
    migrated
    displayName
  }
}
'''

field_name = 'subgraphs'

subgraph_df = query_theGraph_subgraph(subgraph_query, field_name, url, False)

In [5]:
subgraph_df

,id,owner,currentVersion,createdAt,updatedAt,active,migrated,displayName
0,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd-0,{'id': '0x93606b27cb5e4c780883ec4f6b7bed5f6572...,{'id': '0x93606b27cb5e4c780883ec4f6b7bed5f6572...,1608252009,1645807948,False,True,PoolTogether
1,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN,{'id': '0x93606b27cb5e4c780883ec4f6b7bed5f6572...,{'id': 'ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasU...,1608252009,1645807948,False,True,PoolTogether
2,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e-0,{'id': '0x673b6e9fe607f6ddf4a4f25b386b846c5c82...,{'id': '0x673b6e9fe607f6ddf4a4f25b386b846c5c82...,1619441050,1619790992,False,False,PoolTogether
3,Gv1rewdMuEeDQ9sWsWMjJfwKWrTxo5e19vgxr1br7Ut4,{'id': '0x673b6e9fe607f6ddf4a4f25b386b846c5c82...,{'id': 'Gv1rewdMuEeDQ9sWsWMjJfwKWrTxo5e19vgxr1...,1619441050,1619790992,False,False,PoolTogether
4,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e-1,{'id': '0x673b6e9fe607f6ddf4a4f25b386b846c5c82...,{'id': '0x673b6e9fe607f6ddf4a4f25b386b846c5c82...,1619442230,1645807948,False,True,PoolTogether
...,...,...,...,...,...,...,...,...
715,61Z53azfvtkcrWD1Bvz7qXfS1Q1mvCMH3EwaEfLXDhqC,{'id': '0x8f0e2028ab47435eea14bba3d2123c9697cd...,{'id': '61Z53azfvtkcrWD1Bvz7qXfS1Q1mvCMH3EwaEf...,1650880301,1650880301,True,True,mulan market mainnet
716,9tKy8mF93SK3JvThtk4sbSNd8nwt1Hyg5t11Jhh1u893,{'id': '0xef268658776cf63e2d4e732da6cde8f87735...,{'id': '9tKy8mF93SK3JvThtk4sbSNd8nwt1Hyg5t11Jh...,1650898459,1650898459,True,True,Swapr Mainnet
717,3oPKQiPKyD1obYpi5zXBy6HoPdYoDgxXptKrZ8GC3N1N,{'id': '0x4fb90ce4d11d42f9a7967dd4d1d2d5f3908c...,{'id': '3oPKQiPKyD1obYpi5zXBy6HoPdYoDgxXptKrZ8...,1650917017,1650917017,True,True,gnosis-safe-mainnet
718,DMLEJZt2LLoyQ267rFBtWqjKfUdY5dFRFsyfD3RExutv,{'id': '0x0e2897efecc61c7b6a3f1b13e34308cb366b...,{'id': 'DMLEJZt2LLoyQ267rFBtWqjKfUdY5dFRFsyfD3...,1650959777,1651056343,True,True,DODOEX V2 Polygon


In [6]:
subgraph_df['ownerId'] = [i['id'] for i in subgraph_df['owner']]
subgraph_df['version'] = [i['version'] for i in subgraph_df['currentVersion']]
subgraph_df['subgraphDeploymentId'] = [i['subgraphDeployment']['id'] for i in subgraph_df['currentVersion']]
subgraph_df['subgraphid'] = [i['id'] for i in subgraph_df['currentVersion']]
subgraph_df.drop(['owner', 'currentVersion'], axis = 1, inplace = True)
subgraph_df.head()

,id,createdAt,updatedAt,active,migrated,displayName,ownerId,version,subgraphDeploymentId,subgraphid
0,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd-0,1608252009,1645807948,False,True,PoolTogether,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd-0-0
1,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN,1608252009,1645807948,False,True,PoolTogether,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN-0
2,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e-0,1619441050,1619790992,False,False,PoolTogether,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e,0,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e-0-0
3,Gv1rewdMuEeDQ9sWsWMjJfwKWrTxo5e19vgxr1br7Ut4,1619441050,1619790992,False,False,PoolTogether,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e,0,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,Gv1rewdMuEeDQ9sWsWMjJfwKWrTxo5e19vgxr1br7Ut4-0
4,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e-1,1619442230,1645807948,False,True,PoolTogether,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e,0,0xbbde25a2c85f55b53b7698b9476610c3d1202d88870e...,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e-1-0


In [8]:
# function used to query each type of event from The Graph
def query_theGraph(raw_query, field_name, url, trans_type, verbose=False):

    query_parts =raw_query.split(')')
    paginator = ", where: {}" 
    n = 0
    records = []
    while n < 1000:
        if (n == 0):
            # order by blockNumber to get all events, as deafult ordering is by id
            query = 'query '+query_parts[0] + 'orderBy:blockNumber, where:{blockNumber_gt: 0, type:' + trans_type +'})' + query_parts[1]
        else:
            query = 'query '+query_parts[0]+ 'orderBy:blockNumber, where: {blockNumber_gt: ' + str(indexer) + ', type:' + trans_type +'})'+query_parts[1]

        if verbose:
            print(query)

        r = requests.post(url, json = {'query':query})

        try:
            d = json.loads(r.content)['data'][field_name]
        except:
            print(r.content)
            errors = json.loads(r.content)['errors']
            print(errors)
            for e in errors:
                print(e['message'])

        #print(f'results {len(d)}')
        records.extend(d)
        #print(f'total {len(records)}')
        try:
            indexer = d[-1]['blockNumber']
        except:
            print('noblock')
        
        n += 1
        if len(d) < 99:
            break
        
    return pd.DataFrame(records)

In [9]:
#get mintSignal event
url = 'https://api.thegraph.com/subgraphs/name/graphprotocol/graph-network-mainnet'

minted_query = '''query{
  signalTransactions() {
    id
    blockNumber
    signer{
      curator
      {
        id
      }
    }
    signal
    tokens
    type
    subgraphDeployment{
     id
    } 
  }  
}
'''

field_name = 'signalTransactions'
trans_type = 'MintSignal'

curation_mint_df = query_theGraph(minted_query, field_name, url, trans_type, False)

In [13]:
#get MintNSignal event
url = 'https://api.thegraph.com/subgraphs/name/graphprotocol/graph-network-mainnet'

name_mint_query = '''query{
  nameSignalTransactions(){
    id
    blockNumber
    signer{
      curator{
        id
      }
    }
    type
    nameSignal
    versionSignal
    tokens
    subgraph{
      id
      currentVersion
      {
        version
        subgraphDeployment {
          id
        }
      }
      displayName
    }
  }
}
'''

field_name = 'nameSignalTransactions'
trans_type = 'MintNSignal'

name_mint_df = query_theGraph(name_mint_query, field_name, url, trans_type, False)

In [11]:
curation_mint_df['signer'] = [i['curator']['id'] for i in curation_mint_df['signer']]
curation_mint_df['subgraphDeployment']  = [i['id'] for i in curation_mint_df['subgraphDeployment']]
curation_mint_df.head()

,id,blockNumber,signer,signal,tokens,type,subgraphDeployment
0,0x4c1060853752b5221b1e27c2264034d0bd5fd86c5267...,11474121,0xadca0dd4729c8ba3acf3e99f3a9f471ef37b6825,279284800875378823397,78000000000000000000000,MintSignal,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...
1,0xea7dbe2f6164530a53ac8cf9d99b46993f44d8ca8258...,11476852,0xadca0dd4729c8ba3acf3e99f3a9f471ef37b6825,627949250367270370,351147682958369832716,MintSignal,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...
2,0xfbdb37e510284ebbeb45fbfdf28191219faf84bc31bf...,11476875,0xadca0dd4729c8ba3acf3e99f3a9f471ef37b6825,125503147008134479971,86010901965910864972811,MintSignal,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...
3,0xfbda17a348fb979ad36e27445d656b23458a938fadd2...,11476931,0xadca0dd4729c8ba3acf3e99f3a9f471ef37b6825,89477475325111327822,80557400454965249175744,MintSignal,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...
4,0x395da0c992967e085a68fd69825c8f629486ef10d48b...,11531681,0xadca0dd4729c8ba3acf3e99f3a9f471ef37b6825,1568268903568881,975000000000000000,MintSignal,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...


In [14]:
name_mint_df['signer'] = [i['curator']['id'] for i in name_mint_df['signer']]
name_mint_df['subgraphId'] = [i['id'] for i in name_mint_df['subgraph']]
name_mint_df['subgraphDeploymentID'] = [i['currentVersion']['subgraphDeployment']['id'] for i in name_mint_df['subgraph']]
name_mint_df['subgraphversion'] = [i['currentVersion']['version'] for i in name_mint_df['subgraph']]
name_mint_df['subgraphname'] = [i['displayName'] for i in name_mint_df['subgraph']]
name_mint_df.drop(['subgraph'], axis = 1, inplace = True)
name_mint_df.head()

,id,blockNumber,signer,type,nameSignal,versionSignal,tokens,subgraphId,subgraphDeploymentID,subgraphversion,subgraphname
0,0x4c1060853752b5221b1e27c2264034d0bd5fd86c5267...,11474121,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,MintNSignal,279284800875378823397,279284800875378823397,80000000000000000000000,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,0,PoolTogether
1,0xea7dbe2f6164530a53ac8cf9d99b46993f44d8ca8258...,11476852,0x0f3cf18b37c55f5a319d6cae8d8ba114cb094af0,MintNSignal,627949250367270370,627949250367270370,360151469700892136118,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,0,PoolTogether
2,0xfbdb37e510284ebbeb45fbfdf28191219faf84bc31bf...,11476875,0x0f3cf18b37c55f5a319d6cae8d8ba114cb094af0,MintNSignal,125503147008134479971,125503147008134479971,88216309708626528177242,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,0,PoolTogether
3,0xfbda17a348fb979ad36e27445d656b23458a938fadd2...,11476931,0xb6916933aa0bac4b6d09260719c4055363642745,MintNSignal,89477475325111327822,89477475325111327822,82622974825605383769993,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,0,PoolTogether
4,0x395da0c992967e085a68fd69825c8f629486ef10d48b...,11531681,0x849ed8601451dcac8012425ccd56fd1339db498c,MintNSignal,1568268903568881,1568268903568881,1000000000000000000,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,0,PoolTogether


In [19]:
self_curation_list = []
for i, j in curation_mint_df[['subgraphDeployment', 'signer']].iterrows():
    if_true = False
    for k, l in subgraph_df[['subgraphDeploymentId', 'ownerId']].iterrows():
        if l['subgraphDeploymentId'] == j['subgraphDeployment'] and l['ownerId'] == j['signer']:
            if_true = True
    if if_true == True:
        self_curation_list.append('True')
    else:
        self_curation_list.append('False')
print(len(self_curation_list))

7642


In [30]:
self_curation_subgraph_list = []
for i, j in subgraph_df[['subgraphDeploymentId', 'ownerId']].iterrows():
    if_true = False
    for k, l in curation_mint_df[['subgraphDeployment', 'signer']].iterrows():
        if j['subgraphDeploymentId'] == l['subgraphDeployment'] and j['ownerId'] == l['signer']:
            if_true = True
    if if_true == True:
        self_curation_subgraph_list.append('True')
    else:
        self_curation_subgraph_list.append('False')
print(len(self_curation_subgraph_list))

720


In [21]:
curation_mint_df['self_curation'] = self_curation_list
curation_mint_df['self_curation'].value_counts()

False    7640
True        2
Name: self_curation, dtype: int64

In [24]:
subgraph_df['self_curation'] = self_curation_subgraph_list
subgraph_df['self_curation'].value_counts()

False    718
True       2
Name: self_curation, dtype: int64

In [32]:
self_curation_name_subgraph_list = []
for i, j in subgraph_df[['subgraphDeploymentId', 'ownerId']].iterrows():
    if_true = False
    for k, l in name_mint_df[['subgraphDeploymentID', 'signer']].iterrows():
        if j['subgraphDeploymentId'] == l['subgraphDeploymentID'] and j['ownerId'] == l['signer']:
            if_true = True
    if if_true == True:
        self_curation_name_subgraph_list.append('True')
    else:
        self_curation_name_subgraph_list.append('False')
print(len(self_curation_name_subgraph_list))

720


In [33]:
subgraph_df['self_curation_name'] = self_curation_name_subgraph_list
subgraph_df['self_curation_name'].value_counts()

False    402
True     318
Name: self_curation_name, dtype: int64

In [37]:
self_curation_true = subgraph_df[(subgraph_df['self_curation'] == 'True') | (subgraph_df['self_curation_name'] == 'True')]
self_curation_true.to_csv('./self_curation_true.csv')
self_curation_false = subgraph_df[(subgraph_df['self_curation'] == 'False') | (subgraph_df['self_curation_name'] == 'False')]
self_curation_false.to_csv('./self_curation_false.csv')